<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Preprocess-and-create-model" data-toc-modified-id="Preprocess-and-create-model-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Preprocess and create model</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Train</a></span></li></ul></li></ul></div>

In [2]:
from theano.sandbox import cuda
# cuda.use('gpu')

In [9]:
%matplotlib inline
import io
import utils_exp; 
from importlib import reload; reload(utils_exp)
from utils_exp import *
from __future__ import division, print_function

In [10]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [12]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = io.open(path, encoding='utf-8').read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [15]:
text[:100]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all ph'

In [16]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [19]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

In [20]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 58


In [21]:
chars.insert(0, "\0")

In [270]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

In [24]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [25]:
idx = [char_indices[c] for c in text]

In [276]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [274]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

In [42]:
def make_readable(indices):
    return ''.join(indices_char[i] for i in indices)

In [43]:
make_readable(idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [30]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600854


In [44]:
make_readable(sentences[0])

'preface\n\n\nsupposing that truth is a woma'

In [45]:
make_readable(next_chars[0])

'reface\n\n\nsupposing that truth is a woman'

In [50]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [51]:
sentences.shape, next_chars.shape

((600852, 40), (600852, 40))

In [52]:
n_fac = 24

In [53]:
vocab_size

58

In [56]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512,return_sequences=True, dropout=0.2, recurrent_dropout=0.2,
             implementation=2,
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2,
             implementation=2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  import sys


In [55]:
# old interesting to look at the old names
# model=Sequential([
#         Embedding(vocab_size, n_fac, input_length=maxlen),
#         LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size)),
#         Activation('softmax')
#     ])    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, input_shape=(None, 24), dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  import sys


In [57]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [58]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [59]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

/opt/conda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
600852/600852 [==============================] - 673s 1ms/step - loss: 1.5102


In [136]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 24)            1392      
_________________________________________________________________
lstm_5 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_5 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 40, 58)            29754     
_________________________________________________________________
activation_3 (Activation)    (None, 40, 58)            0         
Total para

In [81]:
sentences.shape

(600852, 40)

In [82]:
next_chars.shape

(600852, 40)

In [83]:
np.expand_dims(next_chars,-1).shape

(600852, 40, 1)

In [89]:
next_chars[:,:,np.newaxis].shape

(600852, 40, 1)

In [137]:
foo = np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
model.predict(foo, verbose=0).shape

(1, 40, 58)

In [138]:
model.predict(foo, verbose=0)[0][-1].shape

(58,)

In [60]:
print_example()

ethics is a basic foundation of all that
overpements of the ideal severity. but is, men, and
myself must be unable to the experiences
of a very philosopher on the caste: the believer
of the effect, responsible for in life
to appropriate this
respect to gold and today, a
masy uvertrade possible action, the degree:
it is plabiarity to a sacrifice in the pagche


In [159]:
target = np.expand_dims(next_chars,-1)

In [160]:
target.shape

(600852, 40, 1)

In [140]:
model.fit(sentences, target, batch_size=64, epochs=1)

Epoch 1/1
600852/600852 [==============================] - 663s 1ms/step - loss: 1.2892


In [141]:
print_example()

ethics is a basic foundation of all that had as somewhat is to do what?
     on no more equally against
in religion in which the
worst passions without such a south of galiani) who looks by his fear, are the loftiest acts of the will (and collectors--owing to thee who speak something indicated, and to
humble even it women. in the question what attempts the w


In [142]:
model.optimizer.lr=0.001

In [143]:
model.fit(sentences, target, batch_size=64, nb_epoch=1)

Epoch 1/1
   128/600852 [..............................] - ETA: 14:53 - loss: 1.2555

/opt/conda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


600852/600852 [==============================] - 662s 1ms/step - loss: 1.2554


In [144]:
print_example()

ethics is a basic foundation of all that every things remain who calls life (indignantly and severity, and "immoral" things more
incertain--questionableness,
and novel, a new, but
necessary
to go nothing in
this method: should
in the sacrifice before one is the most troption of others, these later class. he will find itself
problem of fact, the appearance of


In [145]:
model.optimizer.lr=0.0001

In [146]:
model.fit(sentences, target, batch_size=64, nb_epoch=1)

Epoch 1/1
   128/600852 [..............................] - ETA: 14:36 - loss: 1.2645

/opt/conda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


600852/600852 [==============================] - 663s 1ms/step - loss: 1.2358


In [147]:
print_example()

ethics is a basic foundation of all that great. very consciousness of
the logical original problems of improbable as courage, degree and of the
most inccited extend). it may succeed even were
beyond it. man is the changed passionately. there are lead and disguise--but in truth gaily in
itself--we suppose such a hard-hippory--all feeling is
spaniated, educati


In [149]:
mkdir data

In [150]:
model.save_weights('data/char_rnn.h5')

In [151]:
model.optimizer.lr=0.00001

In [152]:
model.fit(sentences, target, batch_size=64, nb_epoch=1)

Epoch 1/1
   128/600852 [..............................] - ETA: 12:21 - loss: 1.2059

/opt/conda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


600852/600852 [==============================] - 666s 1ms/step - loss: 1.2203


In [153]:
print_example()

ethics is a basic foundation of all that have
done by nature. the fundamental certainty of
the life is known, backward pleasure in the formula of his task; a woman does not know, would not
avow whatever there is no higher and highly misunderstood,
retired and schopenhauer with morals, however, because they have open for his "love with the fact that the spiri


In [154]:
model.fit(sentences, target, batch_size=64, nb_epoch=1)

Epoch 1/1
   128/600852 [..............................] - ETA: 11:54 - loss: 1.2354

/opt/conda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


600852/600852 [==============================] - 665s 1ms/step - loss: 1.2078


In [155]:
print_example()

ethics is a basic foundation of all that there was
preferred to be conversant that severity
in this motive; it is a thoughtfulness in a comparison, or even of a
superficiality that is no disposition of an
also the france of truth the victory of the love. as though
conscious of human nerves, already more useful and good-natured, taste? fools, at most mellow m


In [156]:
print_example()

ethics is a basic foundation of all that grades a god let us art a decline of those its intentional interpretation services and animalizations.=--whoever has he
is, appear, hatred, for instance, which perhaps guiltlessness of the
will and self-deluder. there may be it unscientific
society), with the first difficulty opinion, an
around to him from the univers


In [157]:
model.save_weights('data/char_rnn.h5')